In [16]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re
import subprocess

import utilities as ut
from local_jquery import json_query_byid
#import pyjq as jq

#from goog_api import google_api_key
#import googlemaps
#gmaps = googlemaps.Client(key=google_api_key)

In [17]:
# import data
# business, checkin, user, review, tip

business_data = pd.read_json(os.path.join(os.getcwd(), 'data', ut.generate_yelp_name('business')), lines=True)
df = pd.DataFrame.from_dict(business_data)

In [18]:
# filter by state, and by food.
user_state = 'CA'

df = df.loc[df['state'] == user_state]
df = df.loc[df['categories'].str.contains("food|restaurant", na=False, flags=re.IGNORECASE, regex=True)]

In [19]:
# filter categories by the user's food preference
#df.head()
user_prefs = ['Vegan']
user_prefs = "|".join(user_prefs)
df = df.loc[df['categories'].str.contains(user_prefs, na=False, flags=re.IGNORECASE, regex=True)]
df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
921,vLT1KtrA9bWvjFOg-0xVIg,Pieology Pizzeria,"7000 Hollister Ave, Ste F",Goleta,CA,93117,34.430453,-119.872345,3.5,131,0,"{'RestaurantsAttire': ''casual'', 'Alcohol': '...","Restaurants, Gluten-Free, Pizza, Vegan, Fast Food","{'Monday': '0:0-0:0', 'Tuesday': '10:0-20:0', ..."
6300,iHdrLu8deF5GceB0V1rPhA,Oliver's,1198 Coast Village Rd,Santa Barbara,CA,93108,34.421334,-119.644981,4.5,313,1,"{'RestaurantsAttire': 'u'dressy'', 'BikeParkin...","American (Traditional), Vegan, Restaurants","{'Tuesday': '17:0-21:0', 'Wednesday': '17:0-21..."
18784,Js3m_GdqNUGnEgXJ8WMtfQ,Hana Kitchen,503 State St,Santa Barbara,CA,93101,34.416504,-119.695618,3.0,253,1,"{'RestaurantsDelivery': 'True', 'DogsAllowed':...","Bubble Tea, Restaurants, Food, Asian Fusion, V...","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'..."
19072,GBD-YjcOD1nG0wv4L0Lvfg,Venus in Furs,18 E Cota St,Santa Barbara,CA,93101,34.418217,-119.696120,5.0,24,1,"{'BusinessParking': '{'garage': False, 'street...","American (Traditional), Restaurants, Wine Bars...","{'Wednesday': '16:0-22:0', 'Thursday': '16:0-2..."
20444,Ksz_jsyyK3Zjlq_Cz81ZBA,Adama Vegan Comfort Cuisine,428 Chapala St,Santa Barbara,CA,93101,34.415688,-119.696007,4.0,238,0,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...","Restaurants, Vegetarian, Vegan","{'Monday': '17:0-21:0', 'Tuesday': '17:0-21:0'..."


In [20]:
# small utility that writes out all possible categories to a txt file.
# used by chatting system.
#x = list(df['categories'].str.split(', '))
#categories = []
#for lst in x:
#    for i in lst:
#        if i not in categories:
#            categories.append(i)

In [21]:
ut.out_write_category(df, 'categories')

In [22]:
# drop the unused columns
new_df = df[['business_id','name','latitude','longitude','categories','stars', 'review_count']].copy()

In [23]:
# create a new rating column based on the baye's prob of stars and review count.
def set_bayes(R, W):
    def bayesian_probability(rating, review_count):
        return (W*R + review_count * rating) / (W + review_count)
    return bayesian_probability

bayes_prob = set_bayes(2, 3)

new_df['new_rating'] = new_df.apply(lambda row: bayes_prob(row['stars'], row['review_count']), axis=1)
new_df.sort_values('new_rating', ascending=False)
#new_df.to_json('latlon.json',orient="records", lines=True)


,business_id,name,latitude,longitude,categories,stars,review_count,new_rating
150169,izSgTrqebu8bN8ONOCs6cQ,Oat Bakery,34.416548,-119.695626,"Bakeries, Vegan, Specialty Food, Food Delivery...",5.0,123,4.928571
19072,GBD-YjcOD1nG0wv4L0Lvfg,Venus in Furs,34.418217,-119.696120,"American (Traditional), Restaurants, Wine Bars...",5.0,24,4.666667
145143,GuzbBFraIq-fbkjfvaTRvg,Mesa Verde,34.401348,-119.723080,"Garage Door Services, Restaurants, Live/Raw Fo...",4.5,1796,4.495831
127157,XgehM3pBZWuXi45J2dqRJQ,Green Table,34.417872,-119.700702,"Juice Bars & Smoothies, Restaurants, Food, Veg...",4.5,319,4.476708
6300,iHdrLu8deF5GceB0V1rPhA,Oliver's,34.421334,-119.644981,"American (Traditional), Vegan, Restaurants",4.5,313,4.476266
55593,-I7M6l0h46VdiN09Y1NUZw,Apna Indian Kitchen,34.419239,-119.698387,"Gluten-Free, Gastropubs, Vegan, Restaurants, I...",4.5,246,4.469880
50521,UiJiURqivPDuvMsFgRF3yQ,The Honey B,34.419336,-119.695822,"Vegetarian, Gluten-Free, Food, Restaurants, Co...",4.5,215,4.465596
94492,ZJzlns_dFkapVajmdFHGuw,Embermill,34.422053,-119.703135,"Caribbean, Restaurants, Gluten-Free, Cajun/Cre...",4.5,211,4.464953
41668,IZZ7J14VnJGyUqYH1tgT6w,Satellite,34.422616,-119.704029,"Restaurants, Vegan, Tapas/Small Plates, Vegeta...",4.5,159,4.453704
78005,Nbht3URjeV94507do0goEw,Pressed,34.418992,-119.700057,"Ice Cream & Frozen Yogurt, Juice Bars & Smooth...",4.5,111,4.434211


In [9]:
business_ids = new_df.head(3)['business_id'].tolist()

# writes a json file of the relevant reviews based on the top businesses.
lst = {}
once_passed = False

def json_outquery(dat,wt):
    with open(os.path.join(os.getcwd(), 'data', 'last_query.json'), wt) as f:
        f.write(''.join(dat))

for i in business_ids:
    lst[i] = json_query_byid(i, 'business_id')
    if not once_passed:
        json_outquery(lst[i],'w')
        once_passed = True
        continue
    json_outquery(lst[i],'a')

In [10]:
gen_df = pd.read_json(os.path.join(os.getcwd(), 'data', 'last_query.json'), lines=True)
gen_df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,CzaSF8lyivDgINKo60kOxQ,j3sAb6ISO2uhVISy-xDWDA,vLT1KtrA9bWvjFOg-0xVIg,3,0,0,0,The salad was very good pizza was excellent!!!...,2016-09-18 20:59:01
1,wDYXODwk2maHz9_IZe5IyA,jlxneprZ6nQdRRctZ556_g,vLT1KtrA9bWvjFOg-0xVIg,5,0,0,0,"Best pizza in town. Staff is always friendly, ...",2016-09-18 19:26:42
2,Sd-hnPHKJdaCtqLC_rmI5A,vZEppzKSMeAjaus-1jsJFw,vLT1KtrA9bWvjFOg-0xVIg,5,1,1,1,We were visiting in Santa Barbara and stopped ...,2017-10-25 21:15:14
3,rgMZ35JQw9c_KlvzOagO3w,LntVAVzW4FqBlDSaZm4HXA,vLT1KtrA9bWvjFOg-0xVIg,3,1,0,0,"Going to rethink stopping here, again. We hav...",2018-01-10 21:37:04
4,iED-XZWJBhrGlV5YEofhNQ,4BEIGsz4VhHArD20H7X0mg,vLT1KtrA9bWvjFOg-0xVIg,5,0,0,0,Some of the best service! Thank you Brianna an...,2017-04-20 03:07:04


In [11]:
# generate a summary of all reviews related to each business.

from transformers import BartTokenizer, BartForConditionalGeneration

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
def gen_summary(text):
    inp = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inp, min_length=50, max_length=150, length_penalty=2.0, num_beams=2, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [12]:
final_df = new_df.head(3).copy()
summaries = []
for i in business_ids:
    article = '\n'.join(gen_df.loc[gen_df['business_id'] == i]['text'].tolist())
    summaries.append(gen_summary(article))
final_df['summary'] = summaries

In [13]:
final_df.head()

,business_id,name,latitude,longitude,categories,stars,review_count,new_rating,summary
921,vLT1KtrA9bWvjFOg-0xVIg,Pieology Pizzeria,34.430453,-119.872345,"Restaurants, Gluten-Free, Pizza, Vegan, Fast Food",3.5,131,3.466418,"The pizza is good priced, less than $9 and a g..."
6300,iHdrLu8deF5GceB0V1rPhA,Oliver's,34.421334,-119.644981,"American (Traditional), Vegan, Restaurants",4.5,313,4.476266,Olivers is a primarily gluten-free and solely ...
18784,Js3m_GdqNUGnEgXJ8WMtfQ,Hana Kitchen,34.416504,-119.695618,"Bubble Tea, Restaurants, Food, Asian Fusion, V...",3.0,253,2.988281,Hanna Kitchen is a great place for a quick lun...


In [14]:
#df['city'].unique()

array(['Goleta', 'Santa Barbara', 'Carpinteria', 'Montecito',
       'Isla Vista'], dtype=object)